In [2]:
import json
import cv2
import requests
import sys
import pkg_resources
import numpy as np
from symspellpy import SymSpell, Verbosity
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import eli5
from sklearn import metrics

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/

In [12]:
text = open('./ocr_data/wine_text.txt','r',encoding='UTF8')

wordList = []
f = open("./ocr_data/wine_text.txt", 'r')
lines = f.readlines()
for line in lines[1:]:
    sentence = line.split()
    for i in sentence:
      if i.isdigit() is False:
        wordList.append(i)
f.close()

wordCount = {} 

for word in wordList:
    wordCount[word] = wordCount.get(word, 0) + 1 
    keys = sorted(wordCount.keys())

f = open("/anaconda/envs/azureml_py36/lib/python3.6/site-packages/symspellpy/wordCount.txt", 'w')
for key in keys:
  count = wordCount[key]
  data = key + " " + str(count) +"\n"
  f.write(data)
f.close()

In [13]:
with open("/anaconda/envs/azureml_py36/lib/python3.6/site-packages/symspellpy/wordCount.txt", 'r') as f:
    lines = f.readlines()
    print(lines)

["'the 1\n", 'acre 2\n', 'alesia 2\n', 'alexande 1\n', 'amor 1\n', 'anderson 1\n', 'apa 1\n', 'au 1\n', 'bond 1\n', 'bourgogne 1\n', 'buehler 4\n', 'c/s 1\n', 'cabernet 3\n', 'chablis 1\n', 'chardonnay 1\n', 'cruz 1\n', 'del 1\n', 'felettig 1\n', 'grenache 2\n', 'h 1\n', 'hated 1\n', 'herri 1\n', 'hundred 2\n', "hunter' 1\n", 'jadot 1\n', 'joie 1\n', 'kayli 2\n', 'la 2\n', 'louis 1\n', 'michael 1\n', 'mina 1\n', 'mountain 1\n', 'muse 1\n', 'napa 1\n', 'ndel 1\n', 'no 1\n', 'noir 2\n', 'non 1\n', 'oak 2\n', 'pajarito 1\n', 'paradis 1\n', 'peter 1\n', 'pinot 3\n', 'profuga 1\n', 'qua 1\n', 'quella 1\n', 'r 1\n', 'rhys 2\n', 'river 1\n', 'romanee-conti 4\n', 'sa 1\n', 'santa 1\n', 'sauvignon 2\n', 'silver 2\n', 'sine 1\n', 'subir 1\n', 'syra 1\n', 'syrah 2\n', 'ussian 1\n', 'uvignon 1\n', 'valley 5\n', 'verite 2\n', 'vineyards 2\n', 'vineyards,n 1\n', 'vineyards,r 1\n', 'white 2\n', 'ziehharmoniks 1\n', 'zinfa 1\n', 'zinfandel 1\n']


In [28]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "wordCount.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [164]:
LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024
LIMIT_BOX = 40

rest_api_key = '6f71d5adcbc2cae7d6b38363a188f06c'

def kakao_ocr_resize(image_path):

    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX)/ max(height,width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path,image)

        return image_path
    return None

def kakao_ocr(image_path: str, appkey: str):
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'
    headers = {'Authorization' : 'KakaoAK {}'.format(appkey)}
    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"image":data})

In [19]:
goods_cd =[9321076, 9329911, 9356504, 9356511, 9371088, 9377141, 9400054, 
          9406636, 9416239, 9432628, 9436879, 9436886, 9436893, 9440920, 
          9441026, 9441033, 9441163, 9448940, 9449282, 9466333, 9484009, 
          9484023, 9484047, 9484054, 9484580, 9484597]

In [67]:
from glob import glob

for j in goods_cd:
    img_list = glob('./ocr_data/goods_cd/'+ str(j) + '/*.jpg')
    img_list += glob('./ocr_data/goods_cd/'+ str(j) + '/*.JPG')

    k_fold = 5
    total_size = len(img_list)
    fraction = 1/k_fold
    seg = int(total_size * fraction)

    for i in range(k_fold):
        trll = 0
        trlr = i*seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
    
        train_left_indices = list(img_list[trll: trlr])
        train_right_indices = list(img_list[trrl: trrr])
        train_indices = train_left_indices + train_right_indices
        val_indices = list(img_list[vall: valr])
    
        with open('./ocr_data/fold_data/train_' + str(j) + '_' + str(i) + '.txt', 'w') as f:
            f.write('\n'.join(train_indices) + '\n')
    
        with open('./ocr_data/fold_data/val_' + str(j) + '_' + str(i) + '.txt', 'w') as f:
            f.write('\n'.join(val_indices) + '\n')

In [122]:
for k in range(5):

    train_data = pd.DataFrame(columns=['ID', 'TEXT', 'LABEL'])
    val_data = pd.DataFrame(columns=['ID', 'TEXT', 'LABEL'])

    for j in goods_cd:

        with open('./ocr_data/fold_data/train_'+ str(j) + '_' + str(k) +'.txt', 'r') as f:
            for line in f:
                image_path = line
                resize_impath = kakao_ocr_resize(str.strip(image_path))

                if resize_impath is not None:
                    image_path = resize_impath

                output = kakao_ocr(str.strip(image_path), rest_api_key).json()

                word_arr = []

                if output is not None:
                    result = output['result']

                for i in range(0,len(result)):
                    word = result[i]['recognition_words'][0].lower()
                    word_arr.append(word)
                    
                detectList = []

                for word in word_arr:
                    input_term = word
                    suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)

                    if len(suggestions) != 0:
                        for suggestion in suggestions:
                            detectList.append(suggestion.term)
                    else:
                        detectList.append(word)

                test_string = ""
                for i in detectList:
                    test_string += i + " "
                
                train_data = train_data.append({"TEXT" : test_string, "LABEL" : str(j)}, ignore_index=True)

        with open('./ocr_data/fold_data/val_'+ str(j) + '_' + str(k) +'.txt', 'r') as f:
            for line in f:
                image_path = line
                resize_impath = kakao_ocr_resize(str.strip(image_path))

                if resize_impath is not None:
                    image_path = resize_impath

                output = kakao_ocr(str.strip(image_path), rest_api_key).json()

                word_arr = []

                if output is not None:
                    result = output['result']

                for i in range(0,len(result)):
                    word = result[i]['recognition_words'][0].lower()
                    word_arr.append(word)
                    
                detectList = []

                for word in word_arr:
                    input_term = word
                    suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)

                    if len(suggestions) != 0:
                        for suggestion in suggestions:
                            detectList.append(suggestion.term)
                    else:
                        detectList.append(word)

                test_string = ""
                for i in detectList:
                    test_string += i + " "
                
                val_data = train_data.append({"TEXT" : test_string, "LABEL" : str(j)}, ignore_index=True)
        
    train_data.to_excel('./ocr_data/dataset/'+ 'train_' + str(k) +'.xlsx')
    train_data.to_excel('./ocr_data/dataset/'+ 'val_' + str(k) +'.xlsx')

./ocr_data/goods_cd/9321076/9321076_ (1).jpg



KeyError: 'result'

In [153]:
#1

train = pd.read_excel('./ocr_data/dataset/train_0.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(train['TEXT'])
logit0 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit0.fit(X_train_text, train['LABEL'])

val = pd.read_excel('./ocr_data/dataset/val_0.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_test_text = text_transformer.fit_transform(val['TEXT'])
y_pred = logit0.predict(X_test_text)
y_true = val['LABEL']

print(metrics.classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

     9321076     0.9859    0.9459    0.9655        74
     9329911     1.0000    0.7609    0.8642        46
     9356504     0.8769    0.9194    0.8976        62
     9356511     0.8235    0.9032    0.8615        31
     9371088     0.7400    0.7255    0.7327        51
     9377141     0.3158    1.0000    0.4800        24
     9400054     1.0000    0.9231    0.9600        39
     9406636     1.0000    0.4444    0.6154         9
     9416239     1.0000    1.0000    1.0000         8
     9432628     1.0000    0.8889    0.9412        72
     9436879     1.0000    1.0000    1.0000         5
     9436886     1.0000    1.0000    1.0000         6
     9436893     1.0000    1.0000    1.0000         5
     9440920     1.0000    0.9500    0.9744        20
     9441026     0.9667    0.9062    0.9355        32
     9441033     0.9667    0.9355    0.9508        31
     9441163     1.0000    0.8438    0.9153        32
     9448940     1.0000    

In [154]:
eli5.show_weights(estimator=logit0, 
                  feature_names= list(text_transformer.get_feature_names()),
                 top=(50, 5))

In [149]:
#1

train = pd.read_excel('./ocr_data/dataset/train_1.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(train['TEXT'])
logit1 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit1.fit(X_train_text, train['LABEL'])

val = pd.read_excel('./ocr_data/dataset/val_1.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_test_text = text_transformer.fit_transform(val['TEXT'])
y_pred = logit1.predict(X_test_text)
y_true = val['LABEL']

print(metrics.classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

     9321076     0.9595    0.9467    0.9530        75
     9329911     1.0000    0.8000    0.8889        45
     9356504     0.9194    0.9048    0.9120        63
     9356511     0.8235    0.9032    0.8615        31
     9371088     0.7800    0.7647    0.7723        51
     9377141     0.3117    1.0000    0.4752        24
     9400054     1.0000    0.9487    0.9737        39
     9406636     0.5556    0.5000    0.5263        10
     9416239     1.0000    1.0000    1.0000         8
     9432628     1.0000    0.8986    0.9466        69
     9436879     1.0000    1.0000    1.0000         5
     9436886     1.0000    1.0000    1.0000         6
     9436893     1.0000    1.0000    1.0000         5
     9440920     1.0000    0.9500    0.9744        20
     9441026     0.9394    0.9688    0.9538        32
     9441033     1.0000    0.9032    0.9492        31
     9441163     0.9630    0.8125    0.8814        32
     9448940     1.0000    

In [150]:
eli5.show_weights(estimator=logit1, 
                  feature_names= list(text_transformer.get_feature_names()),
                 top=(50, 5))

In [151]:
#1

train = pd.read_excel('./ocr_data/dataset/train_2.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(train['TEXT'])
logit2 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit2.fit(X_train_text, train['LABEL'])

val = pd.read_excel('./ocr_data/dataset/val_2.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_test_text = text_transformer.fit_transform(val['TEXT'])
y_pred = logit2.predict(X_test_text)
y_true = val['LABEL']

print(metrics.classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

     9321076     0.9859    0.9459    0.9655        74
     9329911     1.0000    0.8222    0.9024        45
     9356504     0.9062    0.9062    0.9062        64
     9356511     0.8235    0.9032    0.8615        31
     9371088     0.9000    0.7059    0.7912        51
     9377141     0.3288    1.0000    0.4948        24
     9400054     1.0000    0.9231    0.9600        39
     9406636     0.5000    0.4545    0.4762        11
     9416239     1.0000    1.0000    1.0000         8
     9432628     1.0000    0.9143    0.9552        70
     9436879     1.0000    1.0000    1.0000         5
     9436886     1.0000    1.0000    1.0000         6
     9436893     1.0000    1.0000    1.0000         5
     9440920     1.0000    0.9500    0.9744        20
     9441026     0.9643    0.8438    0.9000        32
     9441033     0.9091    0.9677    0.9375        31
     9441163     0.9677    0.9375    0.9524        32
     9448940     1.0000    

In [152]:
eli5.show_weights(estimator=logit2, 
                  feature_names= list(text_transformer.get_feature_names()),
                 top=(50, 5))

In [160]:
# 나중에 App 단에서 쓸 내용

def predict_ocr(image_path: str):
    resize_impath = kakao_ocr_resize(image_path)

    if resize_impath is not None:
        image_path = resize_impath

    output = kakao_ocr(image_path, rest_api_key).json()

    word_arr = []

    if output is not None:
        result = output['result']

    for i in range(0,len(result)):
        word = result[i]['recognition_words'][0].lower()
        word_arr.append(word)
        
    detectList = []

    for word in word_arr:
        input_term = word
        suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)

        if len(suggestions) != 0:
            for suggestion in suggestions:
                detectList.append(suggestion.term)
        else:
            detectList.append(word)

    test_string = ""
    for i in detectList:
        test_string += i + " "
    
    raw = {'TEXT': [test_string]}
    data = pd.DataFrame(raw)
    print(data)
    X_test_text = text_transformer.transform(data["TEXT"])
    
    test_preds = logit2.predict(X_test_text)
    test_preds
    
    print(logit2.predict_proba(X_test_text))
    print(np.sum(logit2.predict_proba(X_test_text)))
    test_preds_proba = np.max(logit2.predict_proba(X_test_text))
    test_preds_proba
    
    return test_preds, test_preds_proba

In [11]:
import  xgboost
from xgboost import XGBClassifier

In [12]:
xgb = XGBClassifier()

In [ ]:
#1

train = pd.read_excel('./ocr_data/dataset/train_2.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(train['TEXT'])
logit2 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit2.fit(X_train_text, train['LABEL'])

val = pd.read_excel('./ocr_data/dataset/val_2.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_test_text = text_transformer.fit_transform(val['TEXT'])
y_pred = logit2.predict(X_test_text)
y_true = val['LABEL']

print(metrics.classification_report(y_true, y_pred, digits=4))

In [14]:
xgb.fit(X_train_text, train['LABEL'])

XGBClassifier(objective='multi:softprob')

In [15]:
from sklearn.metrics import classification_report

In [16]:
pred_over = xgb.fit(X_train_text, train['LABEL']).predict(X_test_text)
print(classification_report(y_true, pred_over))

              precision    recall  f1-score   support

     9321076       0.98      0.74      0.85        74
     9329911       0.89      0.67      0.77        46
     9356504       0.80      0.79      0.80        62
     9356511       0.81      0.84      0.83        31
     9371088       0.74      0.57      0.64        51
     9377141       0.17      0.92      0.29        24
     9400054       0.97      0.87      0.92        39
     9406636       0.38      0.33      0.35         9
     9416239       1.00      1.00      1.00         8
     9432628       0.97      0.85      0.90        72
     9436879       1.00      1.00      1.00         5
     9436886       1.00      1.00      1.00         6
     9436893       1.00      1.00      1.00         5
     9440920       1.00      0.90      0.95        20
     9441026       0.97      0.88      0.92        32
     9441033       1.00      0.94      0.97        31
     9441163       1.00      0.69      0.81        32
     9448940       0.91    

In [28]:
range(1, 20, 2)

range(1, 20, 2)

In [30]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, train_test_split

folds = 3
param_comb = 5
params = {
        'max_depth': range(1, 20, 2),
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]
        }

# Param describe
# gamma 트리에서 가지를 추가로 치기 위해 필요한 최소한의 손실 감소 기준. 기준값이 클 수록 모형이 더 단순해진다.(> 0)
# max_depth 트리의 최대 깊이.(> 0)
# min_child_weight 트리에서 가지를 추가로 치기 위해 필요한 최소한의 사례 수.(> 0)
# subsample 각각의 트리를 만들 때 데이터에서 사용할 행(row)의 비율(0 ~ 1)
# colsample_bytree 각각의 트리를 만들 때 데이터에서 사용할 열(column)의 비율(0 ~ 1)

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='f1_micro', n_jobs=4, cv=skf.split(X_train_text, train['LABEL']), 
                                    verbose=3, random_state=1001, return_train_score=bool )


random_search.fit(X_train_text,train['LABEL'])

TypeError: 'float' object cannot be interpreted as an integer

In [24]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([6.10951138, 8.7316459 , 2.83082509, 6.32724675, 7.11398204]), 'std_fit_time': array([0.11310602, 0.37824498, 0.06296471, 0.01477564, 0.20733427]), 'mean_score_time': array([0.06138595, 0.05224315, 0.04766925, 0.05990911, 0.07896503]), 'std_score_time': array([0.00357436, 0.00249634, 0.00048366, 0.00204989, 0.00304312]), 'param_subsample': masked_array(data=[1.0, 0.8, 0.8, 0.9, 0.9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_reg_lambda': masked_array(data=[50.0, 100.0, 100.0, 1.0, 10.0],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[100, 100, 100, 100, 100],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[3.0, 3.0, 10.0, 5.0, 0.5],
             mask=[False, False, False, False, Fa

In [25]:
xgb = XGBClassifier(colsample_bylevel=0.4, colsample_bytree=0.7, learning_rate=0.2,
              max_depth=10, min_child_weight=0.5, objective='multi:softprob',
              reg_lambda=10.0, subsample=0.9)

In [26]:
# basic

pred = xgb.fit(X_train_text, train['LABEL']).predict(X_test_text)
print(classification_report(val['LABEL'], pred))

              precision    recall  f1-score   support

     9321076       0.98      0.76      0.85        74
     9329911       0.97      0.65      0.78        46
     9356504       0.84      0.77      0.81        62
     9356511       0.82      0.90      0.86        31
     9371088       0.71      0.63      0.67        51
     9377141       0.18      1.00      0.31        24
     9400054       1.00      0.85      0.92        39
     9406636       1.00      0.22      0.36         9
     9416239       1.00      1.00      1.00         8
     9432628       0.98      0.85      0.91        72
     9436879       1.00      1.00      1.00         5
     9436886       1.00      1.00      1.00         6
     9436893       1.00      1.00      1.00         5
     9440920       1.00      0.90      0.95        20
     9441026       0.97      0.88      0.92        32
     9441033       1.00      0.94      0.97        31
     9441163       1.00      0.69      0.81        32
     9448940       0.82    

In [27]:
logit0 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit0.fit(X_train_text, train['LABEL'])
print(classification_report(val['LABEL'], logit0.predict(X_test_text)))

              precision    recall  f1-score   support

     9321076       0.99      0.95      0.97        74
     9329911       1.00      0.76      0.86        46
     9356504       0.88      0.92      0.90        62
     9356511       0.82      0.90      0.86        31
     9371088       0.74      0.73      0.73        51
     9377141       0.32      1.00      0.48        24
     9400054       1.00      0.92      0.96        39
     9406636       1.00      0.44      0.62         9
     9416239       1.00      1.00      1.00         8
     9432628       1.00      0.89      0.94        72
     9436879       1.00      1.00      1.00         5
     9436886       1.00      1.00      1.00         6
     9436893       1.00      1.00      1.00         5
     9440920       1.00      0.95      0.97        20
     9441026       0.97      0.91      0.94        32
     9441033       0.97      0.94      0.95        31
     9441163       1.00      0.84      0.92        32
     9448940       1.00    

In [ ]:
#1

train = pd.read_excel('./ocr_data/dataset/train_0.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_train_text = text_transformer.fit_transform(train['TEXT'])
logit0 = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit0.fit(X_train_text, train['LABEL'])

val = pd.read_excel('./ocr_data/dataset/val_0.xlsx', index_col='ID', engine='openpyxl').dropna()
text_transformer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True)
X_test_text = text_transformer.fit_transform(val['TEXT'])
y_pred = logit0.predict(X_test_text)
y_true = val['LABEL']

print(metrics.classification_report(y_true, y_pred, digits=4))